## Data generation for single KMC step
In this notebook, we use the LatGas.py module to generate single step data set for the SR-2 system. In this system, there are two species (fast and slow) and a single vacancy. The vacancy exchange rate of the fast species is fixed to 1.0, while that of the slow species is set to 0.001 to maintain a fast:slow ratio of 1e3.

In [1]:
from onsager import crystal, supercell, cluster
import numpy as np
import LatGas
from tqdm import tqdm
import h5py
import pickle

In [2]:
from scipy.constants import physical_constants
kB = physical_constants["Boltzmann constant in eV/K"][0]

In [3]:
# First, we get the necessary crystal data
with h5py.File("../../CrystalData/CrystData.h5", "r") as fl:
    lattice = np.array(fl["Lattice_basis_vectors"])
    superlatt = np.array(fl["SuperLatt"])
    dxList = np.array(fl["dxList_1nn"])
    NNList = np.array(fl["NNsiteList_sitewise"])
    RtoSiteInd = np.array(fl["RToSiteInd"])
    siteIndtoR = np.array(fl["SiteIndToR"])

# we have 8x8x8 primitive supercells
N_units = superlatt[0,0]
print(N_units)

# Get the nearest neighbors of site 0, the vacancy site.
vacsiteInd = 0
jList = NNList[1:, vacsiteInd]
print(jList)

# Create the FCC supercell
crys = crystal.Crystal(lattice=lattice, basis=[[np.array([0., 0., 0.])]], chemistry=["A"], noreduce=True)
print(crys)
superFCC = supercell.ClusterSupercell(crys, superlatt)
Nsites = len(superFCC.mobilepos)
print(Nsites)

8
[ 15  57 449  71 448  64   7   1   8  56 456 120]
#Lattice:
  a1 = [0.  0.5 0.5]
  a2 = [0.5 0.  0.5]
  a3 = [0.5 0.5 0. ]
#Basis:
  (A) 0.0 = [0. 0. 0.]
512


In [4]:
# Make an initial state
# species 0 is the slow species, species 1 is the fast species, species 2 is the vacancy
# In the lattice gas simulations (SR-2,5 and CR-2,5), the vacancy will always have the largest integer
# as its label

c0 = 0.85
N_0 = int(c0 * (Nsites -1))
print(N_0 / (Nsites-1))

# initially put fast atoms everywhere
initState = np.ones(Nsites, dtype=np.int8)

initState[vacsiteInd] = 2 # vacancy index

# put slow atoms in the first N_0 sites except the vacancy
for site in range(1, N_0 + 1):
    initState[site] = 0

# randomize the occupancies of the non-vacancy sites 
initState[1:] = np.random.permutation(initState[1:])

# check the composition
print(np.unique(initState, return_counts=True))

0.8493150684931506
(array([0, 1, 2], dtype=int8), array([434,  77,   1]))


In [5]:
# Now we do the KMC step and gather the necessary data.

# set the exchange rates for the species
# species 0 is the slow species, species 1 is the fast species
NSpec = 3
SpecRates = np.array([0.001, 1.0])

# Set trajectory parameters
Ntraj = 20000
Nsteps = 1

# Set arrays for collecting the states
units = np.array([N_units, N_units, N_units], dtype=int)
state1Grid = np.zeros((Ntraj, N_units, N_units, N_units), dtype=np.int8)
state2Grid = np.zeros((Ntraj, N_units, N_units, N_units), dtype=np.int8)
state2GridUT = np.zeros((Ntraj, N_units, N_units, N_units), dtype=np.int8)

# set arrays for gathering the KMC data
dispList = np.zeros((Ntraj, NSpec, 3))
rateList = np.zeros(Ntraj)
AllJumpRates_state1 = np.zeros((Ntraj, jList.shape[0]))
AllJumpRates_state2 = np.zeros((Ntraj, jList.shape[0]))

jmpSelects = np.zeros((Ntraj, Nsteps), dtype=int) # which jump was selected
rnChecks = np.zeros((Ntraj, Nsteps)) # random numbers used to select jumps to check later

for traj in tqdm(range(Ntraj), position=0, leave=True, ncols=65):

    # permute the initial state to get a new one
    state = initState.copy()
    state[1:] = np.random.permutation(state[1:]) # leave vacancy site unchanged
    initState = state.copy()
    
    assert initState[0] == NSpec - 1
    
    state1Grid[traj, :, :, :] = LatGas.gridState(state, siteIndtoR, units)
    
    # Then do the trajectory
    X_steps, t_steps, jmpSelectSteps, rn, _ =\
    LatGas.LatGasKMCTraj(state, SpecRates, Nsteps, jList,
                         dxList, vacsiteInd, N_units, siteIndtoR, RtoSiteInd)
    
    state2 = LatGas.gridState(state, siteIndtoR, units)
    state2GridUT[traj, :, :, :] = state2[:, :, :]  # Store the untranslated state - we'll use all for testing
    
    state2Grid[traj, :, :, :] = LatGas.translateState(state2, np.where(state == NSpec-1)[0][0],
                                               vacsiteInd, RtoSiteInd, siteIndtoR, units)
    
    assert state2Grid[traj, 0, 0, 0] == NSpec - 1, "\n{}\n{}".format(np.where(state == NSpec-1),
                                                                     state2Grid[traj, 0, 0, 0])
    
    dispList[traj, :, :] = X_steps[-1]  # Get the end displacements
    
    # get all the rates
    JumpRates_state1 = np.zeros(jList.shape[0])
    for jInd, jSite in enumerate(jList):
        spec = initState[jSite]
        JumpRates_state1[jInd] = SpecRates[spec]
    
    rateList[traj] = 1.0/t_steps[-1]  # Inverse of the time is the rate
    assert np.allclose(1.0/t_steps[-1], np.sum(JumpRates_state1))
    
    AllJumpRates_state1[traj, :] = JumpRates_state1[:]
    
    JumpRates_state2 = np.zeros(jList.shape[0])
    for jInd, jSite in enumerate(jList):
        ci, R = superFCC.ciR(jSite)
        spec = state2Grid[traj, R[0], R[1], R[2]]
        JumpRates_state2[jInd] = SpecRates[spec]
    
    AllJumpRates_state2[traj, :] = JumpRates_state2[:]
    
    jmpSelects[traj,:] = jmpSelectSteps[:]
    rnChecks[traj, :] = rn[:]

100%|████████████████████| 20000/20000 [00:14<00:00, 1400.98it/s]


In [6]:
# Check that the flattened states have the correct species at the correct site.
state1List = state1Grid.reshape(-1, Nsites)
state2List = state2Grid.reshape(-1, Nsites)

for check in tqdm(range(state1List.shape[0]), position=0, leave=True, ncols=65):
    for site in range(Nsites):
        ci, R = superFCC.ciR(site)
        assert state1Grid[check, R[0], R[1], R[2]] == state1List[check, site]
        assert state2Grid[check, R[0], R[1], R[2]] == state2List[check, site]    

100%|█████████████████████| 20000/20000 [00:44<00:00, 452.71it/s]


In [7]:
# Save the data set - commented out right now to prevent replacement
# with h5py.File("Datasets/singleStep_FCC_SR2_c0_{}_Run2.h5".format(int(c0 * 100)), "w") as fl:
#     fl.create_dataset("InitStates", data = state1List)
#     fl.create_dataset("FinStates", data = state2List)
#     fl.create_dataset("SpecDisps", data = dispList)
#     fl.create_dataset("AllJumpRates_Init", data = AllJumpRates_state1)
#     fl.create_dataset("AllJumpRates_Fin", data = AllJumpRates_state2)
#     fl.create_dataset("rates", data = rateList)
#     fl.create_dataset("JumpSelects", data = jmpSelects[:, 0])
#     fl.create_dataset("randNums", data = rnChecks[:, 0])